<a href="https://colab.research.google.com/github/mariamffatima/Smart-Energy-Consumption-Recommender-using-FAISS/blob/LLM-deepseek/LLM_Deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================== Install Dependencies ==================
!pip install -q faiss-cpu sentence-transformers bitsandbytes accelerate transformers peft datasets pandas scikit-learn matplotlib
!pip install -q torch keras transformers

In [ ]:
# ================== Imports ==================
import pandas as pd
import numpy as np
import torch
import faiss
from keras.layers import Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
# ================== Load Dataset ==================
url = "https://huggingface.co/datasets/panda04/smart-home-dataset/raw/main/smart_home_dataset.csv"
data = pd.read_csv(url)

In [ ]:
# ================== Preprocessing ==================
def preprocess_data(df):
    df = df.drop(['Transaction_ID', 'Unix Timestamp'], axis=1)
    df['is_peak_hour'] = df['Hour of the Day'].apply(lambda x: 1 if (6 <= x <= 9) or (18 <= x <= 21) else 0)
    df['part_of_day'] = pd.cut(df['Hour of the Day'], bins=[0, 6, 12, 18, 24], labels=['night', 'morning', 'afternoon', 'evening'])
    df['is_weekend'] = df['Day of the Week'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
    season_dict = {'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
                   'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
                   'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
                   'September': 'Fall', 'October': 'Fall', 'November': 'Fall'}
    df['Season'] = df['Month'].map(season_dict)
    df['hour_sin'] = np.sin(2 * np.pi * df['Hour of the Day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['Hour of the Day'] / 24)
    appliances = ['Television', 'Dryer', 'Oven', 'Refrigerator', 'Microwave']
    df['total_appliance_usage'] = df[appliances].sum(axis=1)
    threshold = df['Energy Consumption (kWh)'].quantile(0.75)
    df['is_high_consumption'] = df['Energy Consumption (kWh)'].apply(lambda x: 1 if x > threshold else 0)
    features_to_scale = ['Line Voltage', 'Voltage', 'Apparent Power', 'Energy Consumption (kWh)']
    df[features_to_scale] = MinMaxScaler().fit_transform(df[features_to_scale])
    for appliance in appliances:
        df[f'{appliance}_efficiency_ratio'] = df[appliance] / (df['Energy Consumption (kWh)'] + 1e-6)
    df['power_factor'] = df['Apparent Power'] / (df['Line Voltage'] * df['Voltage'] + 1e-6)
    df['active_appliances'] = df[appliances].gt(0).sum(axis=1)
    df['energy_per_active_appliance'] = df['Energy Consumption (kWh)'] / (df['active_appliances'] + 1e-6)
    return df

In [ ]:
data = preprocess_data(data)

In [ ]:
# ================== LSTM Forecasting ==================
def prepare_lstm_data(df):
    cat_cols = ['Season', 'part_of_day', 'Day of the Week', 'Offloading Decision']
    df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    X = df_encoded.drop('is_high_consumption', axis=1)
    y = df_encoded['is_high_consumption']
    X['Month'] = X['Month'].map({month: i+1 for i, month in enumerate([
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ])})
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X_train.astype(np.float32), X_val.astype(np.float32), X_test.astype(np.float32), y_train.astype(np.float32), y_val.astype(np.float32), y_test.astype(np.float32)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_lstm_data(data)

In [ ]:
# ⚙️ Safe GPU setup for Colab T4
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled.")
    except RuntimeError as e:
        print("⚠️ Runtime error:", e)
else:
    print("❌ No GPU found.")


⚠️ Runtime error: Physical devices cannot be modified after being initialized


In [ ]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(64, activation='tanh', return_sequences=False, unroll=True))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [ ]:
def create_sequences(X, y, time_steps=24):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
import tensorflow as tf

# Force only this block to run on CPU
with tf.device('/CPU:0'):
    X_train_seq, y_train_seq = create_sequences(X_train.values, y_train.values)
    X_val_seq, y_val_seq = create_sequences(X_val.values, y_val.values)

    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    lstm_model = create_lstm_model(input_shape)

    lstm_model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=10,
        batch_size=32
    )


Epoch 1/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 75s 24ms/step - loss: 0.2156 - mae: 0.3841 - val_loss: 0.1919 - val_mae: 0.3581
Epoch 2/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - loss: 0.1883 - mae: 0.3704 - val_loss: 0.1911 - val_mae: 0.3661
Epoch 3/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - loss: 0.1882 - mae: 0.3727 - val_loss: 0.1915 - val_mae: 0.3598
Epoch 4/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 28s 24ms/step - loss: 0.1887 - mae: 0.3738 - val_loss: 0.1901 - val_mae: 0.3757
Epoch 5/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - loss: 0.1863 - mae: 0.3713 - val_loss: 0.1897 - val_mae: 0.3769
Epoch 6/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 0.1885 - mae: 0.3761 - val_loss: 0.1913 - val_mae: 0.3595
Epoch 7/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 0.1862 - mae: 0.3701 - val_loss: 0.1900 - val_mae: 0.3741
Epoch 8/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - loss: 0.1867 - mae: 0.3731 - val_loss: 0.1898 - val_mae: 0.3776
Epoch 9/10
918/918 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# ================== Embedding ==================
def generate_text_description(row):
    appliances = ['Television', 'Dryer', 'Oven', 'Refrigerator', 'Microwave']
    status = ", ".join(f"{a}: {'ON' if row[a] > 0 else 'OFF'}" for a in appliances)
    return f"At {row['Hour of the Day']}:00 during {row['Season']} {row['part_of_day']}, appliances: {status}. Energy: {row['Energy Consumption (kWh)']:.2f}kWh"

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
data['text_description'] = data.apply(generate_text_description, axis=1)
embeddings = embedding_model.encode(data['text_description'].tolist(), show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1531 [00:00<?, ?it/s]

In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
# ================== DeepSeek LLM Setup ==================
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model in 4-bit quantized format
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Apply LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)
model = get_peft_model(model, lora_config)


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

def generate_recommendation(user_log):
    # 1. Forecast future energy using LSTM (force to CPU)
    with tf.device("/CPU:0"):
        last_vals = X_test.values[-24:].reshape((1, 24, X_test.shape[1]))
        forecast = lstm_model.predict(last_vals)[0][0]

    # 2. Retrieve similar logs via FAISS
    input_text = f"{user_log}, Forecasted Usage: {forecast:.2f}"
    user_embedding = embedding_model.encode([input_text])
    D, I = index.search(user_embedding, 3)
    context = "\n".join(data['text_description'].iloc[i] for i in I[0])

    # 3. Construct prompt
    prompt = f"""### Instruction:
Use the context and forecast to suggest energy optimization actions for the user.

### Context:
{context}

### User Log:
{user_log}

### Forecasted Energy Usage:
{forecast:.2f} kWh

### Response:"""

    # 4. Generate recommendation using DeepSeek model
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
generate_recommendation("The user is using AC and Fan together for 14 hours. What are recommendations for him?")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


"### Instruction:\nUse the context and forecast to suggest energy optimization actions for the user.\n\n### Context:\nAt 23:00 during Summer evening, appliances: Television: OFF, Dryer: OFF, Oven: OFF, Refrigerator: OFF, Microwave: ON. Energy: 0.96kWh\nAt 20:00 during Summer evening, appliances: Television: OFF, Dryer: OFF, Oven: OFF, Refrigerator: OFF, Microwave: ON. Energy: 0.96kWh\nAt 20:00 during Summer evening, appliances: Television: OFF, Dryer: OFF, Oven: OFF, Refrigerator: OFF, Microwave: ON. Energy: 0.34kWh\n\n### User Log:\nThe user is using AC and Fan together for 14 hours. What are recommendations for him?\n\n### Forecasted Energy Usage:\n0.26 kWh\n\n### Response: The user is using AC and Fan together for 14 hours. This is during the off-peak hours when the energy usage is lower. So, it's logical to suggest that the user reduce the energy usage during this period.\n\nBut, I need to consider the context provided.\n\nThe user is in a summer evening, which is typically a high-